<a href="https://colab.research.google.com/github/mwojcik01/BSI-Mateusz-Wojcik/blob/master/Sie%C4%87_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importowanie bibliotek

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard

np.set_printoptions(precision=6, suppress=True)

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
!unzip -uq "/content/drive/My Drive/base_ddsm_cbm_1.zip" -d "./" # wypakowanie plików z Drive

In [6]:
base_dir = './base_ddsm_cbm' # obliczanie ilości plików w każdym folderze
raw_no_of_files = {}
classes = ['ben', 'mal', 'nor']
for dir in classes:
    raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))

raw_no_of_files.items()

dict_items([('ben', 888), ('mal', 1115), ('nor', 9215)])

In [7]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
%%bash
#pwd
cd drive/MyDrive/base_ddsm_cbm
pwd
ls

/content/drive/MyDrive/base_ddsm_cbm
ben
mal
nor


In [ ]:
#kasowanie plików
#%%bash
#pwd
#rm -R images
#ls

In [9]:
data_dir = './images' # tworzenie katalogów: treningowego, walidacyjnego oraz testowego. 
                      #w każdym znajdują się 3 foldery: nor, mal, ben

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_nor_dir = os.path.join(train_dir, 'nor')
train_mal_dir = os.path.join(train_dir, 'mal')
train_ben_dir = os.path.join(train_dir, 'ben')

valid_nor_dir = os.path.join(valid_dir, 'nor')
valid_mal_dir = os.path.join(valid_dir, 'mal')
valid_ben_dir = os.path.join(valid_dir, 'ben')

test_nor_dir = os.path.join(test_dir, 'nor')
test_mal_dir = os.path.join(test_dir, 'mal')
test_ben_dir = os.path.join(test_dir, 'ben')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_nor_dir, train_mal_dir, train_ben_dir, valid_nor_dir, valid_mal_dir, valid_ben_dir, test_nor_dir, test_mal_dir, test_ben_dir]

for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [10]:
#kopiowanie
nor_fnames = os.listdir(os.path.join(base_dir, 'nor'))
mal_fnames = os.listdir(os.path.join(base_dir, 'mal'))
ben_fnames = os.listdir(os.path.join(base_dir, 'ben'))


nor_fnames = [fname for fname in nor_fnames if fname.split('.')[1].lower() in ['png']]
mal_fnames = [fname for fname in mal_fnames if fname.split('.')[1].lower() in ['png']]
ben_fnames = [fname for fname in ben_fnames if fname.split('.')[1].lower() in ['png']]

In [12]:
size = min(len(nor_fnames), len(mal_fnames), len(ben_fnames)) # ustalanie ilości plików w folderze tak, żeby były równe ilości

train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

In [13]:
for i, fname in enumerate(nor_fnames): # proces kopiowania plików
    if i <= train_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(train_nor_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(valid_nor_dir, fname)
        shutil.copyfile(src, dst) 
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(test_nor_dir, fname)
        shutil.copyfile(src, dst)   

for i, fname in enumerate(mal_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'mal', fname)
        dst = os.path.join(train_mal_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'mal', fname)
        dst = os.path.join(valid_mal_dir, fname)
        shutil.copyfile(src, dst) 
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'mal', fname)
        dst = os.path.join(test_mal_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(ben_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'ben', fname)
        dst = os.path.join(train_ben_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'ben', fname)
        dst = os.path.join(valid_ben_dir, fname)
        shutil.copyfile(src, dst) 
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'ben', fname)
        dst = os.path.join(test_ben_dir, fname)
        shutil.copyfile(src, dst)

In [ ]:
print('nor - zbiór treningowy', len(os.listdir(train_nor_dir))) # sprawdzanie ilości plików w folderach
print('nor - zbiór walidacyjny', len(os.listdir(valid_nor_dir)))
print('nor - zbiór testowy', len(os.listdir(test_nor_dir)))

print('mal - zbiór treningowy', len(os.listdir(train_mal_dir)))
print('mal - zbiór walidacyjny', len(os.listdir(valid_mal_dir)))
print('mal - zbiór testowy', len(os.listdir(test_mal_dir)))

print('ben - zbiór treningowy', len(os.listdir(train_ben_dir)))
print('ben - zbiór walidacyjny', len(os.listdir(valid_ben_dir)))
print('ben - zbiór testowy', len(os.listdir(test_ben_dir)))

nor - zbiór treningowy 622
nor - zbiór walidacyjny 177
nor - zbiór testowy 89
mal - zbiór treningowy 622
mal - zbiór walidacyjny 177
mal - zbiór testowy 89
ben - zbiór treningowy 622
ben - zbiór walidacyjny 177
ben - zbiór testowy 89


In [14]:
train_datagen = ImageDataGenerator( # augumentacja danych
    rotation_range=40,      
    rescale=1./255.,
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    shear_range=0.2,        
    zoom_range=0.2,         
    horizontal_flip=True,   
    fill_mode='nearest'     
)

# przeskalowanie o współczynnik 1/255
valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(128, 128),
                                                   batch_size=32,
                                                   class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(128, 128),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 1866 images belonging to 3 classes.
Found 531 images belonging to 3 classes.


In [15]:
model = Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(128, 128, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(units=256, activation='relu'),
        layers.Dense(units=3, activation='softmax') #1 sigmoid, 3 softmax
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       1

In [16]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])


In [17]:
batch_size = 32
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

In [18]:
history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=20,    # 100
                    validation_data=valid_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/20
19/19 [==============================] - 33s 146ms/step - loss: 1.1017 - accuracy: 0.3225 - val_loss: 1.1024 - val_accuracy: 0.2937
Epoch 2/20
19/19 [==============================] - 3s 138ms/step - loss: 1.1038 - accuracy: 0.2959 - val_loss: 1.0968 - val_accuracy: 0.3688
Epoch 3/20
19/19 [==============================] - 3s 139ms/step - loss: 1.0997 - accuracy: 0.2953 - val_loss: 1.0995 - val_accuracy: 0.3187
Epoch 4/20
19/19 [==============================] - 3s 140ms/step - loss: 1.0999 - accuracy: 0.3052 - val_loss: 1.0974 - val_accuracy: 0.3688
Epoch 5/20
19/19 [==============================] - 3s 132ms/step - loss: 1.1020 - accuracy: 0.3251 - val_loss: 1.0980 - val_accuracy: 0.3375
Epoch 6/20
19/19 [==============================] - 3s 137ms/step - loss: 1.0993 - accuracy: 0.3354 - val_loss: 1.0960 - val_accuracy: 0.3562
Epoch 7/20
19/19 [==============================] - 3s 133ms/step - loss: 1.1023 - accuracy: 0.3067 - val_loss: 1.0983 - val_accuracy: 0.3250
Epoch

In [19]:


def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
    fig.show()

plot_hist(history)

